### Import libraries

In [127]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from IPython.display import HTML, display, SVG
from IPython.core import display as ICD
from plotly.offline import init_notebook_mode, iplot

import math
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.regularizers import L1L2
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import model_to_dot

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import svm
from sklearn import tree
from sklearn import neighbors
from sklearn import linear_model

import nltk
import string
from nltk.stem import WordNetLemmatizer, SnowballStemmer

import warnings
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)

In [115]:
seed = 13
stemmer = SnowballStemmer("english")
lemma = WordNetLemmatizer()

In [142]:
SOURCE_FOLDER = 'dataset/'
OUTPUT_FOLDER = 'Models_and_output/'

In [27]:
train_data = pd.read_csv(SOURCE_FOLDER + 'train.csv', encoding='ISO-8859-1')
test_data = pd.read_csv(SOURCE_FOLDER + 'test.csv', encoding='utf8')
test_data = test_data[test_data.columns[1:-1]]

In [30]:
train_data.shape

(1157, 9)

In [39]:
train_data = train_data.dropna(how='any')

In [47]:
X_train = train_data['TRANS_CONV_TEXT']
Y_train = train_data['Patient_Tag']
Y_train = to_categorical(Y_train)

In [95]:
def preprocess_data(X_train, stemmer, lemma):
    preprocessed_data = []
    trans = str.maketrans('/(){}', ' ' * 5)
    trans_punc = str.maketrans('', '', string.punctuation)

    for text in X_train:
        text = text.lower().translate(trans)
        text = text.translate(trans_punc)
        text = [lemma.lemmatize(word) for word in text.split()]
        preprocessed_data.append(' '.join(text))

    return preprocessed_data

In [96]:
preprocessed_data = preprocess_data(X_train, stemmer, lemma)

In [107]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english', ngram_range=(1, 3))
vectorizer.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=1000, min_df=2,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [110]:
X_train = vectorizer.transform(X_train).toarray()
X_train.shape

(926, 1000)

In [116]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.33, random_state=seed)

In [117]:
X_train.shape

(620, 1000)

In [118]:
X_valid.shape

(306, 1000)

In [120]:
def batch_generator_shuffle(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    np.random.shuffle(index)
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:]
        y_batch = y_data[index_batch,:]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            np.random.shuffle(index)
            counter=0

In [119]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

@as_keras_metric
def auc_pr(y_true, y_pred, curve='PR'):
    return tf.metrics.auc(y_true, y_pred, curve=curve)

In [125]:
custom_adam = keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

model = Sequential()
model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dense(128, activation='relu'))
model.add(Dense(2,
                activation='softmax',
                kernel_regularizer=L1L2(l1=0.0, l2=0.4)))

In [130]:
precision = as_keras_metric(tf.metrics.precision)
recall = as_keras_metric(tf.metrics.recall)

model.compile(optimizer=custom_adam,
              loss='categorical_crossentropy',
              metrics=[auc_pr, 'accuracy'])

model.fit_generator(generator=batch_generator_shuffle(X_train, y_train, 32),
                      epochs=7, validation_data=(X_valid, y_valid),
                      steps_per_epoch=X_train.shape[0] / 32)

scores = model.evaluate(X_valid, y_valid)
print ('\n')
print (scores)

Epoch 1/7
20/19 [==============================] - 0s 24ms/step - loss: 0.0565 - auc_pr: 0.9499 - acc: 0.9953 - val_loss: 0.5751 - val_auc_pr: 0.9915 - val_acc: 0.8954
Epoch 2/7
20/19 [==============================] - 0s 7ms/step - loss: 0.0390 - auc_pr: 0.9860 - acc: 0.9969 - val_loss: 0.5648 - val_auc_pr: 0.9870 - val_acc: 0.8758
Epoch 3/7
20/19 [==============================] - 0s 6ms/step - loss: 0.0302 - auc_pr: 0.9855 - acc: 0.9984 - val_loss: 0.7946 - val_auc_pr: 0.9850 - val_acc: 0.8758
Epoch 4/7
20/19 [==============================] - 0s 6ms/step - loss: 0.0201 - auc_pr: 0.9838 - acc: 1.0000 - val_loss: 0.6968 - val_auc_pr: 0.9837 - val_acc: 0.8791
Epoch 5/7
20/19 [==============================] - 0s 6ms/step - loss: 0.0114 - auc_pr: 0.9828 - acc: 1.0000 - val_loss: 0.6145 - val_auc_pr: 0.9829 - val_acc: 0.8824
Epoch 6/7
20/19 [==============================] - 0s 6ms/step - loss: 0.0091 - auc_pr: 0.9824 - acc: 1.0000 - val_loss: 0.5275 - val_auc_pr: 0.9830 - val_acc: 0.87

In [132]:
X_test = test_data['TRANS_CONV_TEXT']
X_test = preprocess_data(X_test, stemmer, lemma)
X_test = vectorizer.transform(X_test).toarray()

In [135]:
output = model.predict(X_test, 32)

patient_tag = np.argmax(output, axis=1)
index = list(range(1, len(output)+1))
test_data_df = pd.DataFrame({'Index': index,'Patient_Tag': patient_tag}).set_index('Index')
test_data_df.to_csv('output.csv', columns=['Patient_Tag'] )

In [140]:
test_data_df.head()

,Patient_Tag
Index,
1,0
2,0
3,0
4,1
5,0


In [144]:
model_json = model.to_json()
with open(OUTPUT_FOLDER + "1_model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights(OUTPUT_FOLDER + "1_model.h5")